<a href="https://colab.research.google.com/github/akshithamaddi/SIGIR/blob/main/topic_modeling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6MB 5.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=575d20fb17a8eca3641de3e0e1774aa468cd9f977922d7042d79e7df4bed8131
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
pip install "gensim==3.8.1"

     |████████████████████████████████| 24.2MB 118kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd


main = pd.read_csv('Article_information.csv',encoding='ISO-8859-1')
header = ["Abstract"]
main.to_csv('output.csv', columns = header)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import Word


ps = PorterStemmer()
lemma = WordNetLemmatizer()


pd.read_csv('output.csv', encoding='ISO-8859-1')
df = pd.read_csv('output.csv',encoding='ISO-8859-1')


df.columns = ['','Abstract']
#lowercase and removing special characters and punctuation marks
df['Abstract'] = df['Abstract'].str.lower()
df['Abstract']= df['Abstract'].str.replace(r'[^\w\s]+','')
#removing numbers
df['Abstract'] = df['Abstract'].str.replace('\d+', '')
#removing stopwords
#stop = stopwords.words('english')
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['inform','inform retriev']
stopwords.extend(newStopWords)
df['Abstract'] = df['Abstract'].apply(lambda x: [item for item in str.split(x) if item not in stopwords])
#lemmatization
df['Abstract']= df['Abstract'].apply(lambda x : [lemma.lemmatize(y) for y in x])
#stemming
df['Abstract']= df['Abstract'].apply(lambda x : [ps.stem(y) for y in x])

def remove_punc(text):
  no_punc=" ".join([c for c in text if c not in string.punctuation])
  return no_punc
df['Abstract']= df['Abstract'].apply(lambda x : remove_punc(x))

#Adding columns for clean_content and clean_title in a csv
csv_input = pd.read_csv('output.csv')
csv_input['clean_abstract'] = df['Abstract']
csv_input.to_csv('output_review.csv', index=False)

In [ ]:
#topic Modeling
import pandas as pd
topicm = pd.read_csv("output_review.csv",encoding="ISO-8859-1")
header = ["clean_abstract"]
topicm.to_csv('clean_abstract.csv', columns = header)

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import nltk;
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
data = topicm.clean_abstract.values.tolist()

data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])


['introduct prospect provid acm inform storag retriev symposium held univers maryland april symposium sponsor univers maryland nation aeronaut space administr special interest group inform retriev sigir acm focu advanc techniqu comput orient technolog inform retriev earli develop statu recent effort document retriev quesitonansw data manag system review briefli']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['introduct', 'prospect', 'provid', 'acm', 'inform', 'storag', 'retriev', 'symposium', 'held', 'univers', 'maryland', 'april', 'symposium', 'sponsor', 'univers', 'maryland', 'nation', 'aeronaut', 'space', 'administr', 'special', 'interest', 'group', 'inform', 'retriev', 'sigir', 'acm', 'focu', 'advanc', 'techniqu', 'comput', 'orient', 'technolog', 'inform', 'retriev', 'earli', 'develop', 'statu', 'recent', 'effort', 'document', 'retriev', 'quesitonansw', 'data', 'manag', 'system', 'review', 'briefli']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['introduct', 'prospect', 'provid', 'acm', 'inform', 'storag', 'retriev', 'symposium', 'held', 'univers', 'maryland', 'april', 'symposium', 'sponsor', 'univers', 'maryland', 'nation', 'aeronaut', 'space', 'administr', 'special', 'interest', 'group', 'inform', 'retriev', 'sigir', 'acm', 'focu', 'advanc', 'techniqu', 'comput', 'orient', 'technolog', 'inform', 'retriev', 'earli', 'develop', 'statu', 'recent', 'effort', 'document', 'retriev', 'quesitonansw', 'data', 'manag', 'system', 'review', 'briefli']


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['introduct', 'acm', 'inform', 'retriev', 'symposium', 'hold', 'symposium', 'sponsor', 'special', 'interest', 'group', 'advanc', 'comput', 'develop', 'system']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('acm', 1),
  ('advanc', 1),
  ('comput', 1),
  ('develop', 1),
  ('group', 1),
  ('hold', 1),
  ('inform', 1),
  ('interest', 1),
  ('introduct', 1),
  ('retriev', 1),
  ('special', 1),
  ('sponsor', 1),
  ('symposium', 2),
  ('system', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.275*"question" + 0.218*"answer" + 0.060*"character" + 0.048*"reflect" + '
  '0.041*"cluster" + 0.026*"close" + 0.022*"phase" + 0.022*"despit" + '
  '0.021*"pose" + 0.019*"mainli"'),
 (1,
  '0.304*"network" + 0.130*"aspect" + 0.109*"link" + 0.062*"far" + '
  '0.060*"languag" + 0.046*"power" + 0.037*"total" + 0.029*"write" + '
  '0.022*"multipl" + 0.019*"wellknown"'),
 (2,
  '0.346*"page" + 0.158*"ad" + 0.155*"prior" + 0.043*"advantag" + 0.031*"hard" '
  '+ 0.006*"insid" + 0.004*"grant" + 0.004*"bear" + 0.000*"web" + '
  '0.000*"target"'),
 (3,
  '0.250*"understand" + 0.094*"vector" + 0.094*"better" + 0.088*"space" + '
  '0.080*"input" + 0.070*"interfac" + 0.041*"yield" + 0.041*"map" + '
  '0.030*"hierarch" + 0.030*"profession"'),
 (4,
  '0.136*"research" + 0.091*"item" + 0.055*"high" + 0.045*"rate" + '
  '0.040*"limit" + 0.039*"level" + 0.034*"share" + 0.032*"usual" + '
  '0.032*"public" + 0.028*"attent"'),
 (5,
  '0.120*"user" + 0.095*"search" + 0.070*"system" + 0.058*"inform

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.843457079872005

Coherence Score:  0.3413808738414733


In [ ]:
coherence = []
perplexity = []
for k in range(5,100):
  print('Round: '+str(k))
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           chunksize=10000,
                                           passes=10,
                                           iterations=200,
                                          eval_every = None) 
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')                                              
  coherence.append([k,coherence_model_lda.get_coherence()])
  doc_lda = lda_model[corpus]
  perplexity.append(lda_model.log_perplexity(corpus))


Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12
Round: 13
Round: 14
Round: 15
Round: 16
Round: 17
Round: 18
Round: 19
Round: 20
Round: 21
Round: 22
Round: 23
Round: 24
Round: 25
Round: 26
Round: 27
Round: 28
Round: 29
Round: 30
Round: 31
Round: 32
Round: 33
Round: 34
Round: 35
Round: 36
Round: 37
Round: 38
Round: 39
Round: 40
Round: 41
Round: 42
Round: 43
Round: 44
Round: 45
Round: 46
Round: 47
Round: 48
Round: 49
Round: 50
Round: 51
Round: 52
Round: 53
Round: 54
Round: 55
Round: 56
Round: 57
Round: 58
Round: 59
Round: 60
Round: 61
Round: 62
Round: 63
Round: 64
Round: 65
Round: 66
Round: 67
Round: 68
Round: 69
Round: 70
Round: 71
Round: 72
Round: 73
Round: 74
Round: 75
Round: 76
Round: 77
Round: 78
Round: 79
Round: 80
Round: 81
Round: 82
Round: 83
Round: 84
Round: 85
Round: 86
Round: 87
Round: 88
Round: 89
Round: 90
Round: 91
Round: 92
Round: 93
Round: 94
Round: 95
Round: 96
Round: 97
Round: 98
Round: 99


In [ ]:
print(coherence)
print(perplexity)

[[5, 0.32131398837018005], [6, 0.34301851181097326], [7, 0.3451323112711918], [8, 0.3293579091776484], [9, 0.3411289976267073], [10, 0.3281008209746063], [11, 0.332500323842521], [12, 0.318448798197049], [13, 0.3198709412226448], [14, 0.31834598371697403], [15, 0.33149041792703043], [16, 0.3441529287192103], [17, 0.321715433287995], [18, 0.32893686329521044], [19, 0.3309625364809664], [20, 0.3323248678618932], [21, 0.33237236273972653], [22, 0.33237028769892585], [23, 0.331920888627938], [24, 0.32961443314650185], [25, 0.3189399655719911], [26, 0.3331210186183314], [27, 0.3287528374918496], [28, 0.33032113665457496], [29, 0.3361795979871357], [30, 0.3220062384841357], [31, 0.3343880730054535], [32, 0.3201087729732998], [33, 0.32496997717492443], [34, 0.33652466320643354], [35, 0.321501305777392], [36, 0.34001088012353214], [37, 0.3340026124227133], [38, 0.32797731957662396], [39, 0.3263346186989653], [40, 0.3224764111923283], [41, 0.3293182949427298], [42, 0.3226347819348991], [43, 0.3

In [ ]:
column_names = ["num topics","Coherence"]
df = pd.DataFrame(coherence, columns=column_names)
df['perplexity'] = perplexity
df

,num topics,Coherence,perplexity
0,5,0.321314,-6.740454
1,6,0.343019,-6.744426
2,7,0.345132,-6.774197
3,8,0.329358,-6.770044
4,9,0.341129,-6.775337
...,...,...,...
90,95,0.311322,-7.323945
91,96,0.310553,-7.313562
92,97,0.310528,-7.330532
93,98,0.314008,-7.344770


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['Coherence'], name="coherence"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['perplexity'], name="perplexity"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Coherence and preplexity"
)


# Set x-axis title
fig.update_xaxes(title_text="Num topics")

# Set y-axes titles
fig.update_yaxes(title_text="Coherence", secondary_y=False)
fig.update_yaxes(title_text="Perplexity", secondary_y=True)
fig.update_layout(
    legend=dict(
        x=0.82,
        y=.9,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    )
)

fig.show()

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.395397  0.178263       1        1  13.910937
7     -0.108784 -0.314560       2        1   9.476957
11    -0.083984 -0.146381       3        1   7.407996
8     -0.040976 -0.033337       4        1   6.712738
0     -0.054977 -0.056131       5        1   6.272380
4      0.037271  0.024454       6        1   5.721683
17     0.007024  0.010997       7        1   5.248509
9      0.012409  0.013614       8        1   5.162654
3      0.051395  0.028756       9        1   5.067592
15     0.019462  0.017374      10        1   5.043146
18     0.022414  0.018442      11        1   4.928044
14     0.014535  0.015402      12        1   4.251141
16     0.048055  0.027710      13        1   3.965609
12     0.053157  0.029468      14        1   3.654350
2      0.049744  0.028274      15        1   3.330029
5      0.067971  0.031902      16        1   3.042234
10     0.074456  0.032162      17        1   2.609759
6      0.066688  0.031708      18        1   2.273354
19     0.075514  0.032037      19        1   1.472488
13     0.084023  0.029848      20        1   0.448400, topic_info=           Term         Freq        Total Category  logprob  loglift
118      search  2231.000000  2231.000000  Default  30.0000  30.0000
77         base  1036.000000  1036.000000  Default  29.0000  29.0000
91          use   789.000000   789.000000  Default  28.0000  28.0000
181       model   798.000000   798.000000  Default  27.0000  27.0000
177       learn   693.000000   693.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
432        news     0.031098     2.060442  Topic20  -8.1102   1.2137
362   selection     0.031098     2.060517  Topic20  -8.1102   1.2137
338      result     0.031098     2.060678  Topic20  -8.1102   1.2136
254    approach     0.031098     2.060868  Topic20  -8.1102   1.2135
1141      aware     0.031098     2.061404  Topic20  -8.1102   1.2132

[783 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
55       12  0.967634       access
238      11  0.965637     adaptive
10       17  0.769928    ambiguity
75       15  0.981412     analysis
230       4  0.974421  application
...     ...       ...          ...
83       15  0.955664        value
155      12  0.956409       vector
145       3  0.973655       visual
214       1  0.976466       weight
74        3  0.983293         word

[139 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 12, 9, 1, 5, 18, 10, 4, 16, 19, 15, 17, 13, 3, 6, 11, 7, 20, 14])